### Compilations

In [ ]:
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

### Dataset

In [ ]:
DATASET='Fluo-N2DH-SIM+'

In [32]:
%cd /workdir
!rm -r {DATASET}
!unzip {DATASET}.zip

/workdir
Archive:  Fluo-N2DH-SIM+.zip
   creating: Fluo-N2DH-SIM+/01/
   creating: Fluo-N2DH-SIM+/01/images/
  inflating: Fluo-N2DH-SIM+/01/images/t000.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t001.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t002.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t003.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t004.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t005.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t006.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t007.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t008.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t009.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t010.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t011.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t012.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t013.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t014.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t015.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t016.tif  
  inflating: Fluo-N2DH-SIM+/01/images/t017.tif  
  inflati

In [35]:
%cd /workdir/SW/RotationDetection/track_by_detection
!python convert2obb.py \
--dataset {DATASET} \
--lineage 01 \
--path_root /workdir

/workdir/SW/RotationDetection/track_by_detection
Namespace(augment=False, augment_size=None, consider_mitoses=False, copy_tra2seg=False, dataset='Fluo-N2DH-SIM+', lineage='01', path_root='/workdir')
100%|███████████████████████████████████████████| 65/65 [00:04<00:00, 14.18it/s]


In [ ]:
# change train and test paths inside the script
%cd /workdir/SW/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert_ann2dota.py

In [38]:
# change root path inside the script
%cd /workdir/SW/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert2jpg.py /workdir/{DATASET}/02/images .tif 1

/workdir/SW/RotationDetection/dataloader/dataset/UFRGS_CELL
Converting images to jpg: 100%|███████████████| 150/150 [00:02<00:00, 65.91it/s]
/workdir/Fluo-N2DH-SIM+/02/images
MEAN:  23.154136710273022
STD:  8.492522183409303


In [52]:
# change train path inside the script
%cd /workdir/SW/RotationDetection/dataloader/dataset/UFRGS_CELL
!python data_crop.py /workdir/{DATASET}/02

/workdir/SW/RotationDetection/dataloader/dataset/UFRGS_CELL
class_list 1
find image 150
find label 150
100%|█████████████████████████████████████████| 150/150 [02:33<00:00,  1.02s/it]


In [ ]:
# change test path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python txt2xml.py \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/annotations/dota_format' \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/annotations/xml_rotdet' \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/images'

In [42]:
import os
from glob import glob

os.makedirs(f'/workdir/{DATASET}/crop_all/images', exist_ok=True)
os.makedirs(f'/workdir/{DATASET}/crop_all/labeltxt', exist_ok=True)

path_images = glob(f'/workdir/{DATASET}/*/crop/images/*.png')
path_labels = glob(f'/workdir/{DATASET}/*/crop/labeltxt/*.xml')

for path in path_images:
    src = path
    name = os.path.split(path)[-1]
    name = path.split('/')[3]+'_'+name
    
    dst = os.path.join('/workdir',DATASET,'crop_all','images',name)
    
    os.rename(src, dst)
    
for path in path_labels:
    src = path
    name = os.path.split(path)[-1]
    name = path.split('/')[3]+'_'+name
    
    dst = os.path.join('/workdir',DATASET,'crop_all','labeltxt',name)
    
    os.rename(src, dst)

In [43]:
%cd /workdir/SW/RotationDetection/dataloader/dataset/
!python convert_data_to_tfrecord.py --VOC_dir=/workdir/{DATASET}/crop_all \
                                    --xml_dir='labeltxt' \
                                    --image_dir='images' \
                                    --save_name='train'  \
                                    --img_format='.png'  \
                                    --save_dir=/workdir \
                                    --dataset='UFRGS_CELL_1class'

/workdir/SW/RotationDetection/dataloader/dataset
/workdir/SW/RotationDetection

100%|██████████████████████████████████████| 2985/2985 [00:26<00:00, 111.07it/s]

Conversion is complete!


## Train

In [ ]:
#!rm -r /workdir/msc/RotationDetection/output/**/*Fluo-N2DH-SIM+*

In [ ]:
# change the path to the detector!
%cd /workdir/SW/RotationDetection/tools/r2cnn
!python train.py

/workdir/SW/RotationDetection/tools/r2cnn
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

/workdir/SW/RotationDetection



Instructions for updating:
Please switch to tf.train.get_or_create_global_step



tfrecord path is --> /workdir/UFRGS_CELL_1class_train*

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(

## Test

| Model      | AP50 normal | AP50 mitoses | P50 normal | R50 normal | P50 mitoses | R50 mitoses |   AP50    |   AP75    |   AP50:95   |
|----------- | ----------- | ------------ | ---------- | ---------- | ----------- | ----------- |   -----   |   -----   |   -------   |
| RetinaNet  |   75.25     |   56.47      | 37.64      | 85.00      | 10.99       | 82.85       |   65.86   |   19.17   |   29.56     |
| R3Det      |   74.97     |   63.21      | 32.68      | 83.42      | 14.35       | 84.76       |   69.09   |   17.91   |   30.89     |
| R3Det DCL  |   75.50     |   62.63      | 31.58      | 83.54      | 15.75       | 84.76       |   69.06   | **20.57** |   30.73     |
| DCL        |   71.81     |   63.35      | 39.58      | 81.62      | 16.50       | 80.95       |   67.58   |   17.74   |   28.48     |
| CSL        |   72.42     |   53.98      | 35.20      | 84.10      | 10.94       | 84.76       |   63.20   |   18.59   |   27.00     |
| R2CNN      | **77.33**   | **69.65**    | **67.49**  | 82.41      | **52.79**   | 80.95       | **73.49** |   17.30   |   29.71     |
| Refine RetinaNet | 75.15 |   66.75      | 27.93      | **85.68**  | 11.94       | **89.52**   |   70.95   |   18.86   | **31.00**   |
| RSDet      |   74.33     |   63.60      | 38.65      | 83.31      | 9.63        | 87.62       |   68.97   |   16.15   |   29.86     |

In [62]:
%cd /workdir/SW/RotationDetection/tools/r2cnn
!python test_ufrgscell.py --img_dir='/workdir/Fluo-N2DH-SIM+/02/crop/images'  \
                          --gpu=0 \
                          --image_ext='.png' \
                          --test_annotation_path='/workdir/Fluo-N2DH-SIM+/02/crop/labeltxt' \
                          none

/workdir/SW/RotationDetection/tools/r2cnn
/workdir/SW/RotationDetection
model restore from : /workdir/SW/RotationDetection/output/trained_weights/R2CNN_Fluo-N2DH-SIM+-all_smooth_l1_loss/UFRGS_CELL_1class_37299model.ckpt

**********
Restored model
/workdir/SW/RotationDetection/output/trained_weights/R2CNN_Fluo-N2DH-SIM+-all_smooth_l1_loss/UFRGS_CELL_1class_37299model.ckpt
**********
Eval image t029_0261_0227: 100%|████████████| 2400/2400 [01:48<00:00, 22.14it/s]
********************
rotation eval:
Threshold:  0.5
cls : normal_cell|| Recall: 0.8548566347760667 || Precison: 0.9158596602635339|| AP: 0.8044326603736205
mAP is : 0.8044326603736205


^C
Traceback (most recent call last):
  File "test_ufrgscell.py", line 106, in <module>
    tester.eval()
  File "test_ufrgscell.py", line 100, in eval
    test_annotation_path=self.args.test_annotation_path)
  File "../../libs/val_libs/voc_eval_r.py", line 324, in voc_evaluate_detections
    ap,prec,rec = self.do_python_eval(test_imgid_list, tes

## Inference

In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

path_labels = '/workdir/datasets/msc/UFRGS_CELL_2classes/test/annotations/dota_format'
path_draws = '/workdir/msc/RotationDetection/tools/gwd/test_ufrgscell/RetinaNet_UFRGS_CELL_smooth_l1_loss/ufrgscell_img_vis'

path_labels = sorted(glob(os.path.join(path_labels, '*.txt')))
path_draws  = sorted(glob(os.path.join(path_draws, '*.jpg')))

for i in range(len(path_labels)):
    path_ann = path_labels[i]
    path_img = path_draws[i]

    img = cv2.imread(path_img)[:,:,::-1]
    h,w,c = img.shape

    labels = []
    for lb in open(path_ann, 'r').read().split('\n')[2:-1]:
        lb = np.reshape(list(map(float, lb.split(' ')[:-2])), (-1,2))
        labels.append(lb)
    labels = np.int0(labels)
    
    draw = np.copy(img)
    cv2.drawContours(draw, labels, -1, (255,0,0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(draw)

In [ ]:
DATASET='Fluo-N2DL-HeLa'
LINEAGE='02'

!cp -r /workdir/pepper-experiments/models/msc/datasets/{DATASET}/{LINEAGE}/crop/labeltxt \
/workdir/pepper-experiments/models/msc/ISBI/{DATASET}/{LINEAGE}/crop